# Scraping articles by John Ackerman

In [20]:
import time
import re

import pandas as pd
import numpy as np

import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

## Get links of all articles

In [63]:
# Configure driver
profile = webdriver.FirefoxProfile()
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:86.0) Gecko/20100101 Firefox/86.0'
profile.set_preference("general.useragent.override", user_agent)
options = Options()
options.headless = True

# Launch driver
driver = webdriver.Firefox(profile, options=options)
#driver.maximize_window()
driver.get('https://johnackerman.mx/category/la-jornada/')

while True: 
    time.sleep(2)
    try: 
        #Wait for the button "Cargar más" to load
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'gridlove-pagination.gridlove-load-more')))
        
        #Click on "Cargar más" button
        driver.find_element_by_class_name('gridlove-pagination.gridlove-load-more').click()
    
    except:
        print('No more "Cargar más" buttons to click')
        break
        
# Get links of articles
pageSource = driver.page_source
soup = BeautifulSoup(pageSource, 'html.parser')
links = []
for link in soup.find_all('h2', {'class': 'entry-title h3'}):
    x = link.a['href']
    links.append(x)

No more "Cargar más" buttons to click


In [64]:
len(links)

82

## Extract content of each article

In [108]:
def ackerman_content_scrapper(link):
    
    """Get title, date, and body of an article of John Ackerman"""
    
    # Get text of website
    html = urlopen(link)
    soup = BeautifulSoup(html, 'lxml')
    
    if link == ('https://johnackerman.mx/la-unam-de-pie/' or 'https://johnackerman.mx/hegemonia-democratica/'):
        # Get title
        title = soup.find('h1', {'class': 'entry-title'}).get_text()

        # Get date
        date = soup.find('span', {'class': 'updated'}).get_text()

        # Get body of article
        body = soup.find('div', {'class':'entry-content'}).find_all('p')
        texto = []
        for i in range(len(body)):
            parrafo = body[i].get_text()
            texto.append(parrafo)
        texto_unido = '<\n\n> '.join(texto)
    
    elif link == 'https://johnackerman.mx/un-voto-razonado-para-mario-delgado/':
        # Get title
        title = soup.find('h1', {'class': 'entry-title'}).get_text()

        # Get date
        date = soup.find('div', {'class': 'entry-headline h5'}).p.get_text()

        # Get body of article
        body = soup.find_all('p' , {'align': 'JUSTIFY'})
        texto = []
        for i in range(len(body)):
            parrafo = body[i].get_text()
            texto.append(parrafo)
        texto_unido = '<\n\n> '.join(texto)
    
    else: 
        # Get title
        title = soup.find('h1', {'class': 'entry-title'}).get_text()

        # Get date
        date = soup.find('div', {'class': 'entry-headline h5'}).p.get_text()

        # Get body of article
        body = soup.find_all('p' , {'style': 'text-align: justify;'})
        texto = []
        for i in range(len(body)):
            parrafo = body[i].get_text()
            texto.append(parrafo)
        texto_unido = '<\n\n> '.join(texto)
    
    # Return
    return title, date, texto_unido

In [109]:
# Create variables to store information
titles = []
dates = []
bodies = []

In [110]:
# Store information
error_links = []
for link in links:  
    try: 
        title, date, body = ackerman_content_scrapper(link)
        time.sleep(1)
        titles.append(title)
        dates.append(date)
        bodies.append(body)
        
    except AttributeError:
        error_links.append(link)
        print(f'Error in {link}')

Error in https://johnackerman.mx/las-mentiras-de-loret/
Error in https://johnackerman.mx/el-fin-del-partido-tricolor/
Error in https://johnackerman.mx/hegemonia-democratica/


In [111]:
# Format information
data_ackerman = {'title': '', 'date': '', 'body': ''}
data_ackerman['title'] = titles
data_ackerman['date'] = dates
data_ackerman['body'] = bodies
data = pd.DataFrame(data_ackerman)
data['source'] = 'La Jornada'
data.head(10)

,title,date,body,source
0,El uso político del feminismo,"La Jornada, 8 de Marzo de 2021",El reprobable uso político de la causa feminis...,La Jornada
1,El mensaje de Alberto Fernández,"La Jornada, 1º de Marzo de 2021",La visita de Alberto Fernández a México sacudi...,La Jornada
2,Lecciones de Texas,"La Jornada, 22 de Febrero 2021",El mercado energético tejano rebasa los sueños...,La Jornada
3,Soberanía energética,"La Jornada, 15 de Febrero 2021",Las reformas propuestas por el Presidente Andr...,La Jornada
4,"Nuestro reto, crear una “nueva ideología de iz...",Publicada el 10 de Febrero de 2021,El director del Programa Universitario de Estu...,La Jornada
5,Regular las redes,"La Jornada, 8 de Febrero 2021",El principal riesgo para la libertad de expres...,La Jornada
6,Vacunas: adiós a las patentes,"La Jornada, 1º Febrero 2021.",La comunidad internacional está fallando grave...,La Jornada
7,Biden: ¿Peligro para México?,"La Jornada, 25 de enero de 2021",México se benefició con la errática política e...,La Jornada
8,INE censor,"La Jornada, 18 de enero 2021","El pasado viernes, 15 de enero el Instituto Na...",La Jornada
9,La disputa por los organismos autónomos,"La Jornada, 11 de enero 2021.",Por: John M. Ackerman (@JohnMAckerman)<\n\n> E...,La Jornada


In [112]:
data.tail(10)

,title,date,body,source
69,¿Transición o transformación?,"La Jornada, 11 de febrero, 2019",La transformación política en que nos encontra...,La Jornada
70,La batalla por Venezuela,"La Jornada, 28 de enero, 2019",Solamente una persona totalmente desubicada po...,La Jornada
71,"Zapata, Cárdenas, López Obrador","La Jornada, 14 de enero, 2019",Si Andrés Manuel López Obrador logra rescatar ...,La Jornada
72,La credibilidad del Presidente,"La Jornada, 31 de diciembre, 2018","Antes, en el contexto de un gobierno corrupto ...",La Jornada
73,Salarios justos,"La Jornada, 17 de diciembre, 2018",Los salarios estratosféricos de los ministros ...,La Jornada
74,Arranca la Cuarta Transformación,"La Jornada, 2 de diciembre, 2018",Fracasó el proyecto de destrucción institucion...,La Jornada
75,Autoritarismo jurídico,"La Jornada, 5 de noviembre, 2018",Por: John M. Ackerman (@JohnMAckerman)<\n\n> L...,La Jornada
76,"Bienvenida, hermana migrante","La Jornada, 22 de octubre, 2018",Ningún ser humano es ilegal. El hecho de que u...,La Jornada
77,Elena Álvarez-Buylla y la transformación del C...,"La Jornada, 8 de octubre, 2018",La campaña de noticias falsas y manipulación m...,La Jornada
78,"La UNAM, de pie","10 septiembre, 2018",Les salió el tiro por la culata a los provocad...,La Jornada


In [113]:
# Remove links with errors to match length of DataFrame
actual_links = links.copy()
for link in error_links:
    actual_links.remove(link)
len(actual_links)

data['link'] = actual_links
data.insert(0, 'author', 'John Ackerman')
data.tail()

,author,title,date,body,source,link
74,John Ackerman,Arranca la Cuarta Transformación,"La Jornada, 2 de diciembre, 2018",Fracasó el proyecto de destrucción institucion...,La Jornada,https://johnackerman.mx/arranca-la-cuarta-tran...
75,John Ackerman,Autoritarismo jurídico,"La Jornada, 5 de noviembre, 2018",Por: John M. Ackerman (@JohnMAckerman)<\n\n> L...,La Jornada,https://johnackerman.mx/autoritarismo-juridico/
76,John Ackerman,"Bienvenida, hermana migrante","La Jornada, 22 de octubre, 2018",Ningún ser humano es ilegal. El hecho de que u...,La Jornada,https://johnackerman.mx/bienvenida-hermana-mig...
77,John Ackerman,Elena Álvarez-Buylla y la transformación del C...,"La Jornada, 8 de octubre, 2018",La campaña de noticias falsas y manipulación m...,La Jornada,https://johnackerman.mx/elena-alvarez-buylla-y...
78,John Ackerman,"La UNAM, de pie","10 septiembre, 2018",Les salió el tiro por la culata a los provocad...,La Jornada,https://johnackerman.mx/la-unam-de-pie/


In [114]:
data[data.title == 'Un voto razonado para Mario Delgado']

,author,title,date,body,source,link
23,John Ackerman,Un voto razonado para Mario Delgado,"La Jornada, 5 de octubre de 2020.",Hoy inicia el levantamiento de la encuesta fin...,La Jornada,https://johnackerman.mx/un-voto-razonado-para-...


In [115]:
# Save as csv
data.to_csv('../Data/Data_raw/data_Ackerman_LaJornada', index=False)